In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  QuantileTransformer,  KBinsDiscretizer, PowerTransformer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torchmetrics import AUROC
from scipy.stats import zscore
import gc
def seed_everything(seed=1234):
#     random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv')
X_test =  pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv')
train.pop('id')

y = train['target']
train.pop('target')
X_test.pop('id')
X=train

print(X.shape)

del train
gc.collect()

In [ ]:
X_test.shape

In [ ]:
X, X_OOB, y, y_OOB  = train_test_split(X,y,test_size=0.05,random_state=2021,stratify=y)
y = np.float32(y)
y_OOB = np.float32(y_OOB)
print(y_OOB[:100])

In [ ]:
class ClassModel(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.emb = nn.Embedding(160,32)
        self.fc = nn.Linear(100*32,64)
        self.fc2 = nn.Linear(64,32)
        self.dropout = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.5)
        self.out = nn.Linear(32,1)

        torch.nn.init.xavier_uniform_(self.out.weight)
        torch.nn.init.xavier_normal_(self.emb.weight)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)


    def forward(self, x):
        x = F.silu(self.emb(x))
        x = x.view(-1,100*32)
#         x = torch.mean(x,-1)
        x = self.dropout(x)
        x = F.silu(self.fc(x))
        x = self.dropout2(x)
        x = F.silu(self.fc2(x))
        x = torch.sigmoid(self.out(x))

        return x

In [ ]:
def batch_gd(X, X_test, y, X_OOB, y_OOB, epochs):
    qt = QuantileTransformer(n_quantiles=160, output_distribution='normal')
    X = qt.fit_transform(X)
    X_test = qt.transform(X_test)
    bin_cat = KBinsDiscretizer(n_bins=160, encode='ordinal',strategy='uniform')
    X = bin_cat.fit_transform(X)
    X_test = bin_cat.transform(X_test)
    xoob = qt.transform(X_OOB)
    xoob = bin_cat.transform(xoob)

    gc.collect()
    kfold = StratifiedKFold(n_splits = 8, random_state=2021, shuffle=True)
    y_pred = torch.zeros(540000,1)
    metric = AUROC()
    metric_oob = AUROC()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
       # device = 'cpu'
    print('Using {} device'.format(device))   
    y_pred = y_pred.to(device)
    X_test = torch.from_numpy(X_test.astype(np.float32))
    X_test = X_test.long().to(device)
    xoob = torch.from_numpy(xoob.astype(np.float32))
    yoob = torch.from_numpy(y_OOB.astype(np.float32).reshape(-1, 1))
    oob_dataset=torch.utils.data.TensorDataset(xoob.long(),yoob)
    oob_loader = torch.utils.data.DataLoader(oob_dataset, batch_size=64, shuffle=False)
    for idx in kfold.split(X=X, y=y):
        best_metric = 0
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X[train_idx]
        ytrain = y[train_idx]
        xval = X[val_idx]
        yval = y[val_idx]
        
        ytrain = np.array(ytrain)
        yval = np.array(yval)
        ytrain = torch.from_numpy(ytrain.astype(np.float32).reshape(-1, 1))
        yval = torch.from_numpy(yval.astype(np.float32).reshape(-1, 1))
        xtrain = torch.from_numpy(xtrain.astype(np.float32))
        xval = torch.from_numpy(xval.astype(np.float32))
        
        train_dataset=torch.utils.data.TensorDataset(xtrain.long(),ytrain)
        val_dataset=torch.utils.data.TensorDataset(xval.long(),yval)
        
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
        
        model = ClassModel()
#         model.apply(weights_init)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-08)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1,  gamma=0.9)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        model.to(device)
        gc.collect()
        for epoch in range(epochs):
            model.train()
            t0 = datetime.now()
            train_loss = []
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss.append(loss.item())
            train_loss = np.mean(train_loss) 
            if epoch >= 4:
                scheduler.step()
            model.eval()
            with torch.no_grad():
                val_loss = []
                oob_loss = []
                for  inputs, targets in val_loader:
                    inputs, targets =  inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss.append(loss.item())
                    auroc = metric(outputs,targets.int())

                val_loss = np.mean(val_loss)
                for  inputs, targets in oob_loader:
                    inputs, targets =  inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    oob_loss.append(loss.item())
                    auroc_oob = metric_oob(outputs,targets.int())
                oob_loss = np.mean(oob_loss)
                auroc_oob = metric_oob.compute()
                auroc = metric.compute()
                if auroc_oob > best_metric:
                    best_metric = auroc_oob
                    torch.save(model.state_dict(), '/kaggle/working/ckpt_pytorch')
        #         auc_roc = metric(outputs,targets.int()).compute()
                dt = datetime.now() - t0
                print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, \
              Val Loss: {val_loss:.4f},  Val AUC: {auroc:.4f} ,   OOB AUC: {auroc_oob:.4f},   Duration: {dt}')
                auroc = metric.reset()
                auroc_oob = metric_oob.reset()
        with torch.no_grad():
            model = ClassModel()
            model.load_state_dict(torch.load('/kaggle/working/ckpt_pytorch'))
            model.to(device)

            model.eval()
            y_pred += model(X_test)/kfold.n_splits

            del model
            del inputs
            del targets
            del xtrain
            del xval
            del ytrain
            del yval
            del train_dataset
            del val_dataset
            del train_loader
            del val_loader
            gc.collect()

            with torch.cuda.device('cuda:0'):
                torch.cuda.empty_cache()

            torch.cuda.empty_cache()
    return y_pred.cpu().numpy()

In [ ]:
y_pred=batch_gd(X, X_test, y, X_OOB, y_OOB, epochs=15)

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')
sub.iloc[:,1]=y_pred
sub=sub.set_index('id')
sub.to_csv('baseline_pytorch_cv.csv')